## IMPORT LIBRARIES

In [1]:
import os
import pickle
import librosa
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from IPython.display import Audio

import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import * 
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import backend as K

d:\Projects\Hackathons\AI4D ASR Challenge\env\lib\site-packages\lazy_loader\__init__.py:185: RuntimeWarning: subpackages can technically be lazily loaded, but it causes the package to be eagerly loaded even if it is already lazily loaded.So, you probably shouldn't use subpackages with this lazy feature.
  warnings.warn(msg, RuntimeWarning)
d:\Projects\Hackathons\AI4D ASR Challenge\env\lib\site-packages\lazy_loader\__init__.py:185: RuntimeWarning: subpackages can technically be lazily loaded, but it causes the package to be eagerly loaded even if it is already lazily loaded.So, you probably shouldn't use subpackages with this lazy feature.
  warnings.warn(msg, RuntimeWarning)


In [2]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import Input, Dense
# from sklearn.preprocessing import Imputer, StandardScaler


In [3]:
# Directory for loading audios.

os.chdir("D:/Projects/Hackathons/AI4D ASR Challenge/quizes/Swahili-Speech-To-Text/data/train/wav")

### PREPARE DATA

##### Import Audio Data.

In [4]:
rows = []
parent_dir = "./"
files = os.listdir(parent_dir)
for f in files[:1100]:
    audio, fs = librosa.load(f"{f}")
    filename = f.split('.')[0]
    row = {'filename': filename, 'audio': audio}
    rows.append(row)
rows[:5]

[{'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part10',
  'audio': array([0.02857866, 0.03156228, 0.02614644, ..., 0.09533662, 0.09504194,
         0.05807269], dtype=float32)},
 {'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part100',
  'audio': array([ 0.00458592,  0.0062088 ,  0.00575972, ...,  0.01638122,
         -0.00736282, -0.01466544], dtype=float32)},
 {'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part101',
  'audio': array([0.00858414, 0.00945346, 0.00636425, ..., 0.22225821, 0.2822022 ,
         0.10905091], dtype=float32)},
 {'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part102',
  'audio': array([-0.01062333, -0.01205456, -0.0101964 , ..., -0.21636732,
         -0.2038639 , -0.11033028], dtype=float32)},
 {'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part103',
  'audio': array([0.01031982, 0.01360501, 0.01285982, ..., 0.

In [5]:
len(files)

10180

In [6]:
len(rows)

1100

In [7]:
sample_audios = []
for row in rows:
    audio = row['audio']
    sample_audios.append(audio)
sample_audios[:5]

[array([0.02857866, 0.03156228, 0.02614644, ..., 0.09533662, 0.09504194,
        0.05807269], dtype=float32),
 array([ 0.00458592,  0.0062088 ,  0.00575972, ...,  0.01638122,
        -0.00736282, -0.01466544], dtype=float32),
 array([0.00858414, 0.00945346, 0.00636425, ..., 0.22225821, 0.2822022 ,
        0.10905091], dtype=float32),
 array([-0.01062333, -0.01205456, -0.0101964 , ..., -0.21636732,
        -0.2038639 , -0.11033028], dtype=float32),
 array([0.01031982, 0.01360501, 0.01285982, ..., 0.05907759, 0.05388482,
        0.02589668], dtype=float32)]

##### Import Csv Data.

In [8]:
meta_df = pd.read_csv('D:/Projects/Hackathons/AI4D ASR Challenge/quizes/Swahili-Speech-To-Text/metadata.csv')

In [9]:
meta_df.head()

,filename,transcription,filepath,sample_rate,duration
0,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,rais wa tanzania jakaya mrisho kikwete,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.14
1,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,yanayo andaliwa nami pendo pondo idhaa ya kisw...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.10
2,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,inayokutangazia moja kwa moja kutoka jijini da...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.65
3,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,juma hili bara la afrika limeshuhudia raia wa ...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.90
4,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,wakipiga kura ya maoni ilikufanya mabadiliko ya,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,2.94


### EXPLORE OF DATA

In [10]:
txts = []
for row in rows:
    filename = row['filename']
    filter = meta_df[meta_df['filename'] == filename]
    txt = filter[['transcription']].values
    txts.append(txt)

txts[:5]

[array([['rais wa tanzania jakaya mrisho kikwete']], dtype=object),
 array([['yanayo andaliwa nami pendo pondo idhaa ya kiswahili']],
       dtype=object),
 array([['inayokutangazia moja kwa moja kutoka jijini dar es salaam tanzania']],
       dtype=object),
 array([['juma hili bara la afrika limeshuhudia raia wa nchi za niger']],
       dtype=object),
 array([['wakipiga kura ya maoni ilikufanya mabadiliko ya']], dtype=object)]

In [11]:
txts = np.array(txts).reshape(-1)

In [12]:
len(txts)

1100

In [13]:
clean_txts = []
alphabets = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'.split()
for txt in txts:
    clean_txt = []
    for c in txt:
        if c not in alphabets and c != ' ':
            continue
        clean_txt.append(c)
    clean_txt = ''.join(clean_txt)
    clean_txts.append(clean_txt)

In [14]:
clean_txts[:5]

['rais wa tanzania jakaya mrisho kikwete',
 'yanayo andaliwa nami pendo pondo idhaa ya kiswahili',
 'inayokutangazia moja kwa moja kutoka jijini dar es salaam tanzania',
 'juma hili bara la afrika limeshuhudia raia wa nchi za niger',
 'wakipiga kura ya maoni ilikufanya mabadiliko ya']

In [15]:
'' in clean_txts

True

In [16]:
df = pd.DataFrame(clean_txts)
df.columns = ['texts']
df.head()

,texts
0,rais wa tanzania jakaya mrisho kikwete
1,yanayo andaliwa nami pendo pondo idhaa ya kisw...
2,inayokutangazia moja kwa moja kutoka jijini da...
3,juma hili bara la afrika limeshuhudia raia wa ...
4,wakipiga kura ya maoni ilikufanya mabadiliko ya


In [17]:
idxs = df[df['texts'] == ''].index
idxs

Int64Index([19, 21, 56, 584, 655, 710, 814, 822, 873, 922, 1008], dtype='int64')

In [18]:
del clean_txts[idxs[-1]]
del clean_txts[idxs[-2]]
del clean_txts[idxs[-3]]

In [19]:
'' in clean_txts

True

In [20]:
del sample_audios[idxs[-1]]
del sample_audios[idxs[-2]]
del sample_audios[idxs[-3]]

## Data Preprocessing

##### TOKENIZER

In [21]:
def character_dict():
    alphabet = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'
    supported = alphabet.split()

    char_map = {}
    char_map[""] = 0
    char_map["<SPACE>"] = 1
    idx = 2
    for c in supported:
        char_map[c] = idx
        idx += 1
    index_map = {v: k for k, v in char_map.items()}
    return char_map, index_map

In [22]:
char_map, index_map = character_dict()

In [23]:
char_map

{'': 0,
 '<SPACE>': 1,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'j': 11,
 'k': 12,
 'l': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'p': 17,
 'q': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'x': 25,
 'y': 26,
 'z': 27}

In [24]:
def text_to_int_sequence(text):
    """ Convert text to an integer sequence """
    int_sequence = []
    for c in text:
        if c == ' ':
            ch = char_map['<SPACE>']
        elif c in alphabets:
            ch = char_map[c]
        else:
            print(c)
            print('character not found')
            break
        int_sequence.append(ch)
    return np.array(int_sequence)

In [25]:
def int_sequence_to_text(int_sequence):
    """ Convert an integer sequence to text """
    textch = []
    for c in int_sequence:
        ch = index_map[c]
        textch.append(ch)
    text = ''.join(textch)
    text = text.replace('<SPACE>', ' ')
    return text

## DATA GENERATOR

In [26]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, audios, texts, batch_size=32):
        self.audios = audios
        self.texts = texts
        self.batch_size = batch_size
        self.steps = int(len(self.audios) // self.batch_size)
        self.on_epoch_end()

    def __len__(self):
        return self.steps

    def on_epoch_end(self):
        self.indexes = np.arange(self.steps*self.batch_size)
        # np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        indexes = self.indexes[int(index*self.batch_size):int((index+1)*self.batch_size)]
    
        batch_audios = [self.audios[int(i)] for i in indexes]
        batch_texts = [self.texts[int(i)] for i in indexes]
        
        return  self.data_generation(batch_audios, batch_texts)
    
    def data_generation(self, batch_audios, batch_texts):

        longest_audio = max([len(i) for i in batch_audios])
        longest_txt = max([len(i) for i in batch_texts])

        audios          = np.zeros([int(self.batch_size), longest_audio], dtype="float32")
        txts            = np.zeros([int(self.batch_size), longest_txt], dtype="int64")
        audio_length    = np.zeros([int(self.batch_size)], dtype="int64")
        txt_length      = np.zeros([int(self.batch_size)], dtype="int64")

        i = 0
        for audio, txt in zip(batch_audios, batch_texts):

            txt_len = len(txt)

            txt = text_to_int_sequence(txt)
       
            txts[i,: txt_len] = txt

            audio_len = len(audio)

            audios[i, :audio_len] = audio

            audio_length[i] = audio_len
            txt_length[i] = txt_len

            i+=1          
            
        outputs = {'ctc': np.zeros([self.batch_size])}
        inputs = {
                    'the_input':    tf.convert_to_tensor(audios), 
                    'the_labels':   tf.convert_to_tensor(txts), 
                    'input_length': tf.convert_to_tensor(audio_length), 
                    'label_length': tf.convert_to_tensor(txt_length)
                }
        return (inputs, outputs)


In [27]:
dg = DataGenerator(sample_audios, clean_txts)

In [28]:
len(dg)

34

In [29]:
dg[5]

({'the_input': <tf.Tensor: shape=(32, 135387), dtype=float32, numpy=
  array([[ 5.6196041e-05,  1.3418791e-04,  2.4972850e-04, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         [-7.0419593e-04,  1.0930605e-03,  2.0159236e-03, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         [ 2.3599446e-03,  5.4497039e-03,  7.7096177e-03, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         ...,
         [-5.7262392e-04, -1.4827361e-03, -1.2523080e-03, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         [ 1.1664884e-02,  9.6330661e-03,  1.0384502e-02, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
         [ 4.2224964e-04,  3.1806174e-04,  1.2545670e-04, ...,
           0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)>,
  'the_labels': <tf.Tensor: shape=(32, 122), dtype=int64, numpy=
  array([[16,  7, 10, ...,  0,  0,  0],
         [14, 10, 14, ...,  0,  0,  0],
         [10, 14,  6, 

In [30]:
batch1 = dg[0][0]

In [31]:
batch1['the_labels']

<tf.Tensor: shape=(32, 102), dtype=int64, numpy=
array([[19,  2, 10, ...,  0,  0,  0],
       [26,  2, 15, ...,  0,  0,  0],
       [10, 15,  2, ...,  0,  0,  0],
       ...,
       [13,  2, 12, ...,  0,  0,  0],
       [20, 10, 24, ...,  0,  0,  0],
       [12, 22, 13, ...,  0,  0,  0]], dtype=int64)>

In [32]:
batch1['the_input']

<tf.Tensor: shape=(32, 128993), dtype=float32, numpy=
array([[ 0.02857866,  0.03156228,  0.02614644, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00458592,  0.0062088 ,  0.00575972, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00858414,  0.00945346,  0.00636425, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.01875213,  0.02185869,  0.01020589, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.09314458,  0.11669447,  0.11490583, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01866547, -0.02101425, -0.01500233, ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)>

## CTC

In [33]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [34]:
def input_lengths_lambda_func(args):
    input_length = args
    return tf.cast(tf.math.floor(input_length/hop_size)-1, dtype="float32")

In [35]:
def add_ctc_loss(model_builder):
    the_labels      = Input(name='the_labels',      shape=(None,), dtype='float32')
    input_lengths   = Input(name='input_length',    shape=(1,), dtype='float32')
    label_lengths   = Input(name='label_length',    shape=(1,), dtype='float32')

    input_lengths2 = Lambda(input_lengths_lambda_func)(input_lengths)
    if model_builder.output_length:
         output_lengths  = Lambda(model_builder.output_length)(input_lengths2)
    else:
         output_lengths  = input_lengths2
    
    # CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([model_builder.output, the_labels, output_lengths, label_lengths])
    model = Model( inputs=[model_builder.input, the_labels, input_lengths, label_lengths],  outputs=loss_out)
    return model

### MODEL BUILDING:

## LogMelspectrogram

In [36]:
class LogMelSpectrogram(tf.keras.layers.Layer):
    """Compute log-magnitude mel-scaled spectrograms."""

    def __init__(self, sample_rate, fft_size, hop_size, n_mels,
                 f_min=0.0, f_max=None, **kwargs):
        super(LogMelSpectrogram, self).__init__(**kwargs)
        self.sample_rate = sample_rate
        self.fft_size = fft_size
        self.hop_size = hop_size
        self.n_mels = n_mels
        self.f_min = f_min
        self.f_max = f_max if f_max else sample_rate / 2
        self.mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
            num_mel_bins=self.n_mels,
            num_spectrogram_bins=fft_size // 2 + 1,
            sample_rate=self.sample_rate,
            lower_edge_hertz=self.f_min,
            upper_edge_hertz=self.f_max)

    def build(self, input_shape):
        self.non_trainable_weights.append(self.mel_filterbank)
        super(LogMelSpectrogram, self).build(input_shape)

    def call(self, waveforms):
        """Forward pass.
        Parameters
        ----------
        waveforms : tf.Tensor, shape = (None, n_samples)
            A Batch of mono waveforms.
        Returns
        -------
        log_mel_spectrograms : (tf.Tensor), shape = (None, time, freq, ch)
            The corresponding batch of log-mel-spectrograms
        """
        def _tf_log10(x):
            numerator = tf.math.log(x)
            denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
            return numerator / denominator

        def power_to_db(magnitude, amin=1e-16, top_db=80.0):
            """
            https://librosa.github.io/librosa/generated/librosa.core.power_to_db.html
            """
            ref_value = tf.reduce_max(magnitude)
            log_spec = 10.0 * _tf_log10(tf.maximum(amin, magnitude))
            log_spec -= 10.0 * _tf_log10(tf.maximum(amin, ref_value))
            log_spec = tf.maximum(log_spec, tf.reduce_max(log_spec) - top_db)

            return log_spec

        spectrograms = tf.signal.stft(waveforms,
                                      frame_length=self.fft_size,
                                      frame_step=self.hop_size,
                                      pad_end=False)

        magnitude_spectrograms = tf.abs(spectrograms)

        mel_spectrograms = tf.matmul(tf.square(magnitude_spectrograms),
                                     self.mel_filterbank)

        log_mel_spectrograms = power_to_db(mel_spectrograms)

        # add channel dimension
        log_mel_spectrograms = tf.expand_dims(log_mel_spectrograms, 3)

        return log_mel_spectrograms

    def get_config(self):
        config = {
            'fft_size': self.fft_size,
            'hop_size': self.hop_size,
            'n_mels': self.n_mels,
            'sample_rate': self.sample_rate,
            'f_min': self.f_min,
            'f_max': self.f_max,
        }
        config.update(super(LogMelSpectrogram, self).get_config())

        return config

## MODELS

In [37]:
def preprocessin_model(sample_rate, fft_size, frame_step, n_mels, mfcc=False):

    input_data = Input(name='input', shape=(None,), dtype="float32")
    featLayer = LogMelSpectrogram(
        fft_size=fft_size,
        hop_size=frame_step,
        n_mels=n_mels,
        
        sample_rate=sample_rate,
        f_min=0.0,
        
        f_max=int(sample_rate / 2)
    )(input_data)
    
    x = BatchNormalization()(featLayer)
    model = Model(inputs=input_data, outputs=x, name="preprocessin_model")

    return model

In [38]:
def simple_rnn_model(input_dim, output_dim=224):

    input_data = Input(name='the_input', shape=(None, input_dim))
    simp_rnn = GRU(output_dim, return_sequences=True,
                   implementation=2, name='rnn')(input_data)
    y_pred = Activation('softmax', name='softmax')(simp_rnn)
    model = Model(inputs=input_data, outputs=y_pred, name="simple_rnn_model")
    model.output_length = lambda x: x
    return model


In [39]:
def BidirectionalRNN(input_dim, rnn_layers=2, units=400, drop_out=0.5, act='tanh', output_dim=224):

    input_data = Input(name='the_input', shape=(
        None, input_dim))

    x = Bidirectional(LSTM(units,  activation=act,
                      return_sequences=True, implementation=2))(input_data)
    x = BatchNormalization()(x)
    x = Dropout(drop_out)(x)

    for i in range(rnn_layers - 2):
        x = Bidirectional(
            LSTM(units, activation=act, return_sequences=True))(x)
        x = BatchNormalization()(x)
        x = Dropout(drop_out)(x)

    x = Bidirectional(LSTM(units,  activation=act,
                      return_sequences=True, implementation=2))(x)
    x = BatchNormalization()(x)
    x = Dropout(drop_out)(x)

    time_dense = TimeDistributed(Dense(output_dim))(x)

    y_pred = Activation('softmax', name='softmax')(time_dense)

    model = Model(inputs=input_data, outputs=y_pred, name="BidirectionalRNN")

    return model

In [40]:
def conv_rnn(n_mels, output_dim=224, rnn_layers=4, units=400, drop_out=0.5, act='tanh'):

    input_data = Input(name='the_input', shape=(None, n_mels, 1))

    y = Conv2D(32, (3, 3), padding='same')(input_data)  # was 32
    y = Activation('relu')(y)
    y = BatchNormalization()(y)

    x = MaxPooling2D((1, 2))(y)

    x = Conv2D(64, (3, 3), padding='same')(y)  # was 32
    x = Activation('relu')(x)
    y = BatchNormalization()(y)

    x = MaxPooling2D((1, 2))(x)

    x = Conv2D(128, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    y = BatchNormalization()(y)

    x = MaxPooling2D((1, 2))(x)

    x = Dense(128)(x)
    x = Dense(64)(x)
    x = Dense(32)(x)

    x = Reshape((-1, x.shape[-1] * x.shape[-2]))(x)

    for i in range(rnn_layers):
        x = Bidirectional(
            LSTM(units, activation=act, return_sequences=True))(x)
        x = BatchNormalization()(x)
        x = Dropout(drop_out)(x)

    bn_rnn = BatchNormalization()(x)

    time_dense = TimeDistributed(Dense(output_dim))(bn_rnn)

    y_pred = Activation('softmax', name='softmax')(time_dense)

    model = Model(inputs=input_data, outputs=y_pred, name="custom_model")

    return model

In [41]:
def train(model_builder, 
          data_gen,
          epochs, 
          verbose=1,
          optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=0.002, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
          ):    
              
    model = add_ctc_loss(model_builder)

    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    print(model.summary())

    # add checkpointer
    checkpointer = ModelCheckpoint(filepath='models/cnn_rnn.h5', verbose=1)
    early_stopping = EarlyStopping( monitor="val_loss", patience=10, restore_best_weights=True)

    hist = model.fit_generator(generator=data_gen,
                               epochs=epochs,
                               callbacks=[checkpointer, early_stopping], 
                               verbose=verbose, 
                               use_multiprocessing=False)
    
    # save model loss
    with open('/models/cnn_rnn.pickle', 'wb') as f:
        pickle.dump(hist.history, f)

## MODEL TRAINING

In [42]:

sample_rate = 22050
fft_size = 1024
frame_step = 512
n_mels = 128

batch_size = 32
epochs = 3
data_len = len(clean_txts)
output_dim = len(char_map) + 2

In [43]:
dg = DataGenerator(sample_audios, clean_txts, batch_size)

In [44]:
preprocess_model = preprocessin_model(sample_rate, fft_size, frame_step, n_mels)
preprocess_model.summary()

Model: "preprocessin_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, None)]            0         
                                                                 
 log_mel_spectrogram (LogMe  (None, None, 128, 1)      0         
 lSpectrogram)                                                   
                                                                 
 batch_normalization (Batch  (None, None, 128, 1)      4         
 Normalization)                                                  
                                                                 
Total params: 4 (16.00 Byte)
Trainable params: 2 (8.00 Byte)
Non-trainable params: 2 (8.00 Byte)
_________________________________________________________________


In [45]:
speech_model = conv_rnn(n_mels, output_dim = output_dim)
speech_model.summary()

Model: "custom_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None, 128, 1)]    0         
                                                                 
 conv2d (Conv2D)             (None, None, 128, 32)     320       
                                                                 
 activation (Activation)     (None, None, 128, 32)     0         
                                                                 
 batch_normalization_1 (Bat  (None, None, 128, 32)     128       
 chNormalization)                                                
                                                                 
 conv2d_1 (Conv2D)           (None, None, 128, 64)     18496     
                                                                 
 activation_1 (Activation)   (None, None, 128, 64)     0         
                                                      

In [46]:
def build_model(output_dim, custom_model, preprocess_model, calc=None):

    input_audios = Input(name='the_input', shape=(None,))
    pre = preprocess_model(input_audios)
    pre = tf.squeeze(pre, [3])

    y_pred = custom_model(pre)
    model = Model(inputs=input_audios, outputs=y_pred, name="model_builder")
    model.output_length = calc

    return model

In [47]:
model = build_model(output_dim, speech_model, preprocess_model)
model.summary()

Model: "model_builder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None)]            0         
                                                                 
 preprocessin_model (Functi  (None, None, 128, 1)      4         
 onal)                                                           
                                                                 
 tf.compat.v1.squeeze (TFOp  (None, None, 128)         0         
 Lambda)                                                         
                                                                 
 custom_model (Functional)   (None, None, 30)          16249278  
                                                                 
Total params: 16249282 (61.99 MB)
Trainable params: 16241216 (61.96 MB)
Non-trainable params: 8066 (31.51 KB)
_________________________________________________________________


In [48]:
# model.load_weights('cnn_rnn3.h5')

In [50]:
# mlflow.set_experiment('Speech Model-RNN-baseline')
# mlflow.tensorflow.autolog()
hop_size = 512
train(model, dg, epochs=100)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 the_input (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 preprocessin_model (Functi  (None, None, 128, 1)         4         ['the_input[0][0]']           
 onal)                                                                                            
                                                                                                  
 tf.compat.v1.squeeze (TFOp  (None, None, 128)            0         ['preprocessin_model[0][0]']  
 Lambda)                                                                                          
                                                                                            

C:\Users\user\AppData\Local\Temp\ipykernel_3304\1370125407.py:17: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(generator=data_gen,


10/34 [=======>......................] - ETA: 13:03 - loss: 248.2238

InvalidArgumentError: Graph execution error:

Detected at node 'model_1/ctc/CTCLoss' defined at (most recent call last):
    File "C:\Users\user\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\user\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "d:\Projects\Hackathons\AI4D ASR Challenge\env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\Projects\Hackathons\AI4D ASR Challenge\env\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "d:\Projects\Hackathons\AI4D ASR Challenge\env\lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "d:\Projects\Hackathons\AI4D ASR Challenge\env\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asy

## PREDICTION:

In [ ]:
def predict(data_gen,  num_elem=1, index=0):
    
    pred_data = data_gen.__getitem__(index)

    pred_audios = pred_data[0]["the_input"]
    pred_labels = pred_data[0]["the_labels"]
    
    y_pred = model.predict(pred_audios)

    input_shape = tf.keras.backend.shape(y_pred)
    input_length = tf.ones(shape=input_shape[0]) * tf.keras.backend.cast(input_shape[1], 'float32')
    prediction = tf.keras.backend.ctc_decode(y_pred, input_length, greedy=False)[0][0]
    

    for i in range(0, num_elem):  # only on clean data
        
        pred = K.eval(prediction[i]).flatten().tolist()
        pred = [i for i in pred if i != -1]



        ground_truth = int_sequence_to_text(pred_labels[i].numpy())
        hypothesis   = ''.join(int_sequence_to_text(pred))
        # error        = wer(ground_truth, hypothesis)
                
        print('-'*48 + ' ' + str(i) + ' ' + '-'*48)
        print('True transcription:\n' + '\n' + ground_truth)
        print('-'*100)
        print('Predicted transcription:\n' + '\n' + hypothesis)
        # print('-'*100)
        # print('Word Error Rate:' + str(error))
        print('\n')

In [ ]:
rows1 = []
parent_dir1 = "./SWH-05-20101107"
files1 = os.listdir(parent_dir1)
for f in files1:
    audio, fs = librosa.load(f"{parent_dir1}/{f}")
    filename = f.split('.')[0]
    row = {'filename': filename, 'audio': audio}
    rows1.append(row)
rows1[:5]

In [ ]:
sample_audios1 = []
for row in rows1:
    audio = row['audio']
    sample_audios1.append(audio)
sample_audios1[:5]

In [ ]:
meta_df = pd.read_csv('/content/drive/MyDrive/metadata.csv')

In [ ]:
txts1 = []
for row in rows1:
    filename = row['filename']
    filter = meta_df[meta_df['filename'] == filename]
    txt = filter[['transcription']].values
    txts1.append(txt)

txts1[:5]

In [ ]:
txts1 = np.array(txts1).reshape(-1)

In [ ]:
txts1[:5]

In [ ]:
clean_txts1 = []
alphabets = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'.split()
for txt in txts1:
    clean_txt = []
    for c in txt:
        if c not in alphabets and c != ' ':
            continue
        clean_txt.append(c)
    clean_txt = ''.join(clean_txt)
    clean_txts1.append(clean_txt)

In [ ]:
clean_txts1[:5]

In [ ]:
'' in clean_txts1

In [ ]:
dg1 = DataGenerator(sample_audios1[:5], clean_txts1[:5], 5)

In [ ]:
predict(dg1, 4)